In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import os

df = pd.read_csv("Xtern_traindata.csv")
df_copy = df.copy()
print(df)
print(df.shape)

        Year                    Major                University  Time  \
0     Year 2                  Physics  Indiana State University    12   
1     Year 3                Chemistry     Ball State University    14   
2     Year 3                Chemistry         Butler University    12   
3     Year 2                  Biology  Indiana State University    11   
4     Year 3  Business Administration         Butler University    12   
...      ...                      ...                       ...   ...   
4995  Year 2              Mathematics     Ball State University    11   
4996  Year 3                Astronomy  Indiana State University    12   
4997  Year 3                Chemistry         Butler University    13   
4998  Year 3                Astronomy         Butler University    15   
4999  Year 2                Economics  University of Notre Dame    15   

                                                  Order  
0                                  Fried Catfish Basket  
1      

In [26]:
#Changing all of the strings into numerical values
df['Year']=df['Year'].astype('category').cat.codes
df['Major']=df['Major'].astype('category').cat.codes
df['University']=df['University'].astype('category').cat.codes
df['Order']=df['Order'].astype('category').cat.codes

In [27]:
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 

#Since time as inputs, not sure how to exclude Time as its already a number
inputs = ["Year", "Major", "University", "Time"]
output = "Order"
orders_X, orders_Y = df[inputs].values, df[output].values

# Split data 70% into train and 30% into test
x_orders_train, x_orders_test, y_orders_train, y_orders_test = train_test_split(orders_X, 
                                                                                orders_Y, 
                                                                                test_size = 0.30,
                                                                                random_state = 0,
                                                                                stratify=orders_Y)


In [28]:
from sklearn.linear_model import LogisticRegression

reg = 0.1

model = LogisticRegression(C=1/reg, 
                           solver='lbfgs', 
                           multi_class='auto', 
                           max_iter=10000).fit(x_orders_train, y_orders_train)
print (model)

LogisticRegression(C=10.0, max_iter=10000)


In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

predictions = model.predict(x_orders_test)
print('Predicted labels: ', predictions[:30])
print('Actual labels   : ' ,y_orders_test[:30])

print("Overall Accuracy:",accuracy_score(y_orders_test, predictions))
print("Overall Precision:",precision_score(y_orders_test, predictions, average='weighted'))
print("Overall Recall:",recall_score(y_orders_test, predictions, average='weighted'))
mcm = confusion_matrix(y_orders_test, predictions)
print(mcm)

Predicted labels:  [2 8 7 6 3 7 3 5 7 6 5 5 7 3 5 5 7 6 7 7 5 3 8 5 5 5 5 5 9 7]
Actual labels   :  [2 9 7 0 7 7 3 0 7 6 5 5 7 1 5 5 7 0 9 8 6 4 9 0 1 5 4 5 8 7]
Overall Accuracy: 0.44866666666666666
Overall Precision: 0.40079350167966415
Overall Recall: 0.44866666666666666
[[  8   3   8   0   0  20  84   7   4  14]
 [  1  75  13  17   0  32   2  13   0   0]
 [  3  18  74   6   0   2  20   3  18   3]
 [  0   2   0  78   0   1   0  37  24   3]
 [  0  19   0  70   0   5   0  28  11  16]
 [  0   6   2   0   0 131   6   2   1   0]
 [  6   1   7   0   0  23  99   0   3  14]
 [  1   7   2  16   0  26   5  85   4   8]
 [  1  12  14   1   0  11   2  14  67  30]
 [  2   5   4   2   0   5  27  27  23  56]]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

columns = [0,1,2,3]
transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ('preprocess', transformer, columns)])

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', SVC(probability=True))])
model = pipeline.fit(x_orders_train, y_orders_train)

predictions = model.predict(x_orders_test)
prob = model.predict_proba(x_orders_test)
print("Overall Accuracy:",accuracy_score(y_orders_test, predictions))
print("Overall Precision:",precision_score(y_orders_test, predictions, average='weighted'))
print('Average AUC:', roc_auc_score(y_orders_test,prob, multi_class='ovr'))

Overall Accuracy: 0.5573333333333333
Overall Precision: 0.5668067664703302
Average AUC: 0.8946631881091021
